<a href="https://colab.research.google.com/github/sachins301/UTA-Distributed-Computing/blob/main/UTA%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xmltodict

In [1]:
import requests
import xmltodict
import json
import pytz
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import time
import os
import concurrent.futures
import logging


In [10]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Create a custom logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create handlers
# c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('/content/drive/My Drive/Colab Notebooks/utaproject.log', mode='w')
# c_handler.setLevel(logging.DEBUG)
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
# c_format = logging.Formatter('%(asctime)s - %(threadName)s - %(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(threadName)s - %(name)s - %(levelname)s - %(message)s')
# c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
# logger.addHandler(c_handler)
logger.addHandler(f_handler)

# Correctly log messages
logger.info('Logging started')


INFO:__main__:Logging started


In [4]:
def retrieve_data(route_id, onward_calls = False):
  token = userdata.get('TOKEN')
  url = f"http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route={route_id}&onwardcalls={onward_calls}&usertoken={token}"
  response = requests.get(url)
  if response.status_code != 200:
    logger.error(f"Failed to retrieve data. Status code: {response.status_code}")
    return None
  logger.info(f"Data retrieved for route_id: {route_id}")
  xml_data = xmltodict.parse(response.text)
  json_data = json.loads(json.dumps(xml_data))
  return json_data

In [5]:
def convert_to_epoch(timestamp):
    date_part, rest = timestamp.split('.')
    microseconds_part = rest[:-6]  # Extract microseconds part
    timezone_part = rest[-6:]      # Extract timezone part
    microseconds_part = microseconds_part[:6]  # Take only the first 6 digits for microseconds
    timestamp = date_part + '.' + microseconds_part + timezone_part
    dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")
    epoch_time = int(dt.timestamp())

    return str(epoch_time)

In [6]:
def save_data(route_id, json_data):
  file_name = route_id + "-" + convert_to_epoch(json_data["Siri"]["ResponseTimestamp"])  + ".json"
  with open('/content/drive/My Drive/Colab Notebooks/json_dumps/'+file_name, 'w', encoding='utf-8') as f:
    f.write(json.dumps(json_data))
  logger.info(f"Data saved to {file_name}")

In [14]:
route_id_list = ['4', '455', '1', '2', '220']
def process_route(route_id):
  logger.info(f"Processing route_id: {route_id}")
  while True:
    json_data = retrieve_data(route_id)
    if json_data:
      logger.info(f"Saving data for route_id: {route_id}")
      save_data(route_id, json_data)
    time.sleep(5)

In [17]:
# process_route('4')

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_route, route_id_list)

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    logger.info("Shutting down...")

INFO:__main__:Processing route_id: 4
INFO:__main__:Processing route_id: 455
INFO:__main__:Processing route_id: 1
INFO:__main__:Processing route_id: 2
INFO:__main__:Processing route_id: 220
INFO:__main__:Data retrieved for route_id: 4
INFO:__main__:Saving data for route_id: 4
INFO:__main__:Data retrieved for route_id: 455
INFO:__main__:Saving data for route_id: 455
INFO:__main__:Data retrieved for route_id: 1
INFO:__main__:Saving data for route_id: 1
INFO:__main__:Data retrieved for route_id: 2
INFO:__main__:Saving data for route_id: 2
INFO:__main__:Data retrieved for route_id: 220
INFO:__main__:Saving data for route_id: 220
INFO:__main__:Shutting down...


In [ ]:
# No of default workers /cpu available for multithreading
print(os.cpu_count() + 4)

6


In [ ]:
# Opening JSON file
f = open('/content/drive/My Drive/Colab Notebooks/'+file_name)
data = json.load(f)